In [1]:
#*=============================================================================
#| Assignment: ASSIGNMENT 1 - Scheduling network modelling (question-2)
#|
#| Author: Mehulkumar Jariwala (30137326)
 #         Akshaykumar Dhorajiya (30151346)
#| Year: 2022
#| Language: Python
#| To Compile: open the file in the colab and update the csv file at the data variable in the code. Then press ctrl+R to get the output-2.csv file.
#+-----------------------------------------------------------------------------
#|
#| Description: Calculated es, ef, ls, lf, tf, ff and crirical path  
#| Input: Change csv file path at "data = pd.read_csv("Assignment_1.csv")" in code.
#| Output: 1. Program will give total duration of project
#          2. It will give critical path of project
#          3. It will generate output-2.csv file which shows es, ef, ls, lf, tf, ff and activities on critical path in tabular form
#| Algorithm: Following algorithm is used
#             1. Create the class name as node which includes activity, es, ef, ls, lf, tf, ff,previous, next, is_critical
#             2. first create a start node and end node, then create a node for each individual creativity.
#             3.Attach start and end node with first and last activity respectively.
#             4.traverse graph in forward and  backward direction recursively using depth first search technique of "graph" & structure to find the values of es, ef, ls ,lf.
#             6.check the type of relationship,then for given relation R(A,B)
#             7. if A.es +lag >b.es then evaluate value of B.es,B.ef
#             8. if b.es -lag<a.ls then evaluate value of a.ls,a.lf
#.            9.repeat steps 8 and 9 for the fs,sf,ss,and ff realtion.
#.            10. after updating everything, again traverse forward and backward direction recursively using the depth first search technique of graph.
#             11. set count =0.
#.                check wheather node activity is critical or not.
#                 if node is critical then set is critical ="yes"
#                 increament count by value 1.
#.           12. set total critical = count
#.           13 get the edges of critical nodes
#.           14 print critical path
#            15 end
#
#
#                   
#             
#| Known Bugs: NA
#*===========================================================================*/
# ----------------------------------      Question 2      ---------------------------------- 
# importing libraries
import pandas as pd

In [2]:
from __future__ import annotations

# create structure for node
class Node:
    
    def __init__(self,activity,duration):
        self.activity = activity
        self.duration = duration
        self.es = None
        self.ef = None
        self.ls = None
        self.lf = None
        self.tf = None
        self.ff = None
        
        self.previous = []
        self.next = []
        
        #keep "NO" as default value
        self.is_critical = "NO"
    
    def __repr__(self):
        return "{" \
            f"\"Activity\":\"{self.activity}\", " \
            f"\"Duration\":\"{self.duration}\", " \
            f"\"ES\":\"{self.es}\", " \
            f"\"EF\":\"{self.ef}\", " \
            f"\"LS\":\"{self.ls}\", " \
            f"\"LF\":\"{self.lf}\", " \
            f"\"TF\":\"{self.tf}\", " \
            f"\"FF\":\"{self.ff}\", " \
            f"\"previous\":\"{list(map(lambda node: node.activity, self.previous))}\","\
            f"\"next\":\"{list(map(lambda node: node.activity, self.next))}\","\
            f"\"Critical Path\":\"{self.is_critical}\"" \
            "}\n"

In [3]:
# function to generate nodes.
def create_nodes(data):

    # create nodes structure and initialize "start"&"end" nodes as 0
    nodes = dict()
    nodes['Start'] = Node(activity="Start",duration=0)
    nodes['End'] = Node(activity="End",duration=0)
    

    for a in data['Activity']:
        if a not in nodes:
            activity = a
            duration = data[data["Activity"]==a]["Duration"]
            d=duration.values[0] #in order to get exact value of duration
            node = Node(activity=activity,duration=d)
            nodes[activity]=node 
            
    # iterate to keeep track of prev. & next node
    for i,row in data.iterrows():
        current = row['Activity']
        dependents = row['Predecessor'].split(",")
        for d in dependents:
            if d.isalpha():
                nodes[current].previous.append(nodes[d])
                nodes[d].next.append(nodes[current])

    # connect "start" and "end" nodes
    for node in filter(lambda node:node.activity not in ['Start','End'],nodes.values()):
        if not node.next:
            node.next.append(nodes['End'])
            nodes['End'].previous.append(node)
    
        if not node.previous:
            node.previous.append(nodes['Start'])
            nodes['Start'].next.append(node)

    return nodes

**Forward Traversing**

In [4]:
# function for forward traversing
def forward_traverse(end):
    max_es = 0
    node = end
    
    #if predecessor's EF is not defined, go back recursively to find
    #predecessor's EF
    for p in node.previous:
        if p.ef==None:
            forward_traverse(end=p)
        max_es = max(max_es, p.ef)
    
    #ES is the max. value of predecessors EF value
    node.es = max_es
    node.ef = node.es + node.duration

**Backward Traversing**

In [5]:
# Defining function for backward traversing
def backward_traverse(end):
    min_lf = 7777777777777777777
    min_ff = 7777777777777777777
    
    node = end
    
    #if successor's LS is not defined, go further recursively to find
    #successor's LS
    for s in node.next:
        if s.ls == None:
            backward_traverse(end=s)
        
        min_lf = min(min_lf,s.ls)
        min_ff = min(min_lf,s.es - node.ef)
        
    #LF is the min. value of successor's LS value.
    node.lf = min_lf
    node.ls = node.lf - node.duration
    
    #if node is critical, then set is_critical=true
    if node.es - node.ls == 0:
        node.is_critical = "YES"
        node.tf = 0
    
    node.tf = node.ls - node.es
    node.ff = min_ff

**Graph edges, only for critical nodes**

creating (source,destination) pair for critical nodes, by traversing all the nodes

In [6]:
import json

# defining function to get edges of critical nodes
def obtain_edges(nodes):
    edges = [] #create empty list for storing edges
    
    for node in nodes.values():
        src_dict = json.loads(repr(node))
        
        #check if current node is critical, if not then continue to 
        #explore further nodes, and look for the node which is critical
        if src_dict['Critical Path']=="NO":
            continue
        
        src = src_dict['Activity']
        
        #loop through successors(i.e creating the list of successors)
        successors = src_dict['next'].lstrip("[").rstrip("]").split(", ")

        #check whether the list of successors is empty or not.
        if successors != ['']:
            for s in successors:
                s = s.lstrip("'").rstrip("'")
                dest = json.loads(repr(nodes[s]))
                
                #check whether destination is critical or not, if it is 
                #critical then add it into "edges" list
                if dest["Critical Path"]=="YES":
                    edges.append((src,dest["Activity"]))
                    
    return edges

**Find All Corresponding Path**

In [7]:
#defining function to find the paths
def get_path(graph,start,end,path):
    path = path + [start]
    
    if start == end:
        return [path]
    
    if start not in graph:
        return []
    
    paths = []
    
    for node in graph[start]:
        # continue, if node is not discovered
        if node not in path:
            newpaths = get_path(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

**Read Data**

In [8]:
data = pd.read_csv("Assignment_1.csv")
# data

In [9]:
#create nodes
nodes = create_nodes(data)

In [10]:
#Assign "start" and "end" nodes to variable called 'start' and 'end'
start = nodes["Start"]
end = nodes["End"]

In [11]:
#Populate earliest start and finish by recursively call forward function
#from the end node forward_traverse(end=end)
forward_traverse(end=end)

# update end's LS and LF
end.ls = end.lf = end.es

In [12]:
#Populate late start and finish by recursively calling backward function
#from the start node backward_traverse(end=start)
backward_traverse(end=start)

**Check the relationship**

In [13]:
relations_df = data[data[['Relationships','Lag']]['Relationships'].notna()]


relations = list(relations_df['Relationships'])

lags = list(relations_df['Lag'])

for relation in relations:
    relation_type = relation[0:2]
    
    
    #extract relation nodes involved in given relation
    r_nodes = relation[2:].lstrip("(").rstrip(")")
    r_nodes = r_nodes.split(",") #create list of involved nodes

    
    node_a = nodes[r_nodes[0]]
    node_b = nodes[r_nodes[-1]]
    
    lag = int(relations_df[relations_df['Relationships']==relation]['Lag'])
      
    # If relation is SS:
    if relation_type == "SS":
        if node_a.es + lag > node_b.es:
            node_b.es = node_a.es + lag
            node_b.ef = node_b.es + node_b.duration
        if node_b.ls - lag < node_a.ls:
            node_a.ls = node_b.ls - lag
            node_a.lf = node_a.ls + node_a.duration
            
    # If relation is SF
    elif (relation_type == "SF"):
        if (node_a.es + lag > node_b.ef):
            node_b.ef = node_a.es + lag
            node_b.es = node_b.ef - node_b.duration
        if (node_b.lf - lag < node_a.ls):
            node_a.ls = node_b.lf - lag
            node_a.lf = node_a.ls + node_a.duration
            
    # If relation is FS
    elif (relation_type == "FS"):
        if (node_a.ef + lag > node_b.es):
            node_b.es = node_a.ef + lag
            node_b.ef = node_b.es + node_b.duration
        if (node_b.ls - lag < node_a.lf):
            node_a.lf = node_b.ls - lag
            node_a.ls = node_a.lf - node_a.duration
            
    # If relation is FF
    elif (relation_type == "FF"):
        if (node_a.ef + lag > node_b.ef):
            node_b.ef = node_a.ef + lag
            node_b.es = node_b.ef - node_b.duration
        if (node_b.lf - lag < node_a.lf):
            node_a.lf = node_b.lf - lag
            node_a.ls = node_a.lf - node_a.duration

In [14]:
# After updating everything, we will traverse again
start = nodes["Start"]
end = nodes["End"]
forward_traverse(end=end)
end.ls = end.lf = end.es
backward_traverse(end=start)

Check For critical nodes

In [15]:
count = 0

#if node is critical, then set is_critical = "Yes"
for node in nodes.values():
    if node.es - node.ls==0:
        node.is_critical = "YES"
        node.tf = 0
        count = count + 1
    else:
        node.tf = node.ls - node.es

total_critical = count #assign count to variable called "total_critical"

In [16]:
# getting graph edges, only for critical nodes
edges = obtain_edges(nodes)

**create graph**

In [17]:
adjacency = dict()

for (s,d) in edges:
    if s not in adjacency:
        adjacency[s] = {d:1}
    else:
        adjacency[s][d] = 1
            
#add "End" node
adjacency["End"] = {}

graph = adjacency

Find All Corresponding Path

In [19]:
path = []
paths = get_path(graph,"Start","End",path)

#creating path string
path_str = "\n ".join(map(lambda path: " -> ".join(path), paths))
ans2_critical_path = path_str

# Critical Paths
print("\nCritical Path:\n-------------------------------\n", path_str,end=" ")
print("\n-------------------------------\n")
print("total duration of project "+str(end.ls))


Critical Path:
-------------------------------
 Start -> C -> L -> P -> N -> End
 Start -> C -> L -> P -> Q -> S -> End 
-------------------------------

total duration of project 32


**creating resultant dataframe**

In [ ]:
dataframe = []

for node in nodes.values():
    node_str = repr(node)
    node_dict = json.loads(node_str)
    dataframe.append(node_dict)

dataframe = pd.DataFrame(dataframe)

In [ ]:
# printing resultant dataframe
ans2 = dataframe
dataframe.to_csv("ans-2.csv")